In [ ]:
import ants
import os
import glob
import nipype.interfaces.io as nio
import time

GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
current_group = GROUPS[0]
current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
TEMPLATE_IND = '%s_desc-masked_T1w.nii.gz'
TEMPLATE_TMP = '%s_space-MNI152NLin2009cAsym_desc-masked_T1w.nii.gz'

T1_DIR='/output/masked_T1/'+current_group+'/'
TMP_OUT_DIR='/output/group_template/'+current_group+'_group_template/'
if not os.path.exists(TMP_OUT_DIR):
    os.makedirs(TMP_OUT_DIR)

with open(current_sub_list_file, 'r') as f_sub:
    sub_list_raw= f_sub.readlines()
sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 

# dataGrabber
ds_masked_t1 = nio.DataGrabber(infields=['sub_id'])
ds_masked_t1.inputs.base_directory = T1_DIR # database
ds_masked_t1.inputs.template = TEMPLATE_TMP # from cwd
ds_masked_t1.inputs.sort_filelist = True
ds_masked_t1.inputs.sub_id = sub_list
res_masked_t1 = ds_masked_t1.run()
group_files=res_masked_t1.outputs.outfiles
group_images = [ants.image_read(x, dimension = 3) for x in group_files]
print(group_files)
t0=time.time()
btp = ants.build_template(initialTemplate = None,
  image_list = group_images,
  iterations = 4,
  gradient_step = 0.2,
  verbose = True,
  syn_metric = 'CC',
  type_of_transform = 'SyN' )
ants.plot(btp)
print('PD template construction takes: ', time.time()-t0)

In [2]:
group_images

[ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (176, 256, 256)
 	 Spacing    : (1.0, 1.0, 1.0)
 	 Origin     : (86.9031, 110.0642, -127.0)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.],
 ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (192, 256, 256)
 	 Spacing    : (1.0, 1.0, 1.0)
 	 Origin     : (93.2432, 102.0, -127.0)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.],
 ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (192, 256, 256)
 	 Spacing    : (1.0, 1.0, 1.0)
 	 Origin     : (91.0651, 104.24, -127.0)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.],
 ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (160, 256, 256)
 	 Spacing    : (1.0, 1.0, 1.0)
 	 Origin     : (75.6705, 121.708, -112.4722)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.],
 ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : 

In [2]:
help(ants.build_template)

Help on function build_template in module ants.registration.build_template:

build_template(initial_template=None, image_list=None, iterations=3, gradient_step=0.2, **kwargs)
    Estimate an optimal template from an input image_list
    
    ANTsR function: N/A
    
    Arguments
    ---------
    initial_template : ANTsImage
        initialization for the template building
    
    image_list : ANTsImages
        images from which to estimate template
    
    iterations : integer
        number of template building iterations
    
    gradient_step : scalar
        for shape update gradient
    
    kwargs : keyword args
        extra arguments passed to ants registration
    
    Returns
    -------
    ANTsImage
    
    Example
    -------
    >>> import ants
    >>> image = ants.image_read( ants.get_ants_data('r16') , 'float')
    >>> image2 = ants.image_read( ants.get_ants_data('r27') , 'float')
    >>> image3 = ants.image_read( ants.get_ants_data('r85') , 'float')
    >>> timag

In [1]:
# Masking PD/ET/NC group images
import os
import time
from nipype.interfaces import fsl

def dataGraber_sub(SUB_ID, DATA_DIR, TMPT_STR):
    import nipype.interfaces.io as nio
    import time
    t0=time.time()
    print('Grabbing files for: ', SUB_ID)
    OUT_FILE=[]
    out_len=len(TMPT_STR)
    if out_len == 0:
        print(SUB_ID+' has no files named: ', TMPT_STR)
        return OUT_FILE
    else:
        for i in range(out_len):
            TMP='%s/anat/%s_'+TMPT_STR[i]
            ds = nio.DataGrabber(infields=['subject_id', 'subject_id'])
            ds.inputs.base_directory = DATA_DIR # database
            ds.inputs.template = TMP 
            ds.inputs.subject_id = [SUB_ID]
            ds.inputs.sort_filelist = True
            res = ds.run()
            res_list = res.outputs.outfiles
            OUT_FILE.append(res_list)
        #print(SUB_ID+' files: ', OUT_FILE)
        print('dataGraber takes: ', time.time()-t0 )
        return OUT_FILE
## masking all group subjects
GROUPS=['PD','ET','NC']
SUB_LIST=[]; AGE_LIST=[];
OUT_DIR='/output/masked_T1/'

for group_name in GROUPS:
    current_group=group_name
    current_sub_list_file = '/codes/devel/PD_Marker/'+current_group+'_info_ICA.list'
    # create dir for output
    current_OUT_DIR=OUT_DIR+current_group+'/'
    current_ind_OUT_DIR = current_OUT_DIR+'individual/'
    current_tmp_OUT_DIR = current_OUT_DIR+'mni2009cAsym/'
    if not os.path.exists(current_ind_OUT_DIR):
        os.makedirs(current_ind_OUT_DIR)
    if not os.path.exists(current_tmp_OUT_DIR):
        os.makedirs(current_tmp_OUT_DIR)
    #read sub list
    with open(current_sub_list_file, 'r') as f_sub:
        sub_list_raw= f_sub.readlines()
    sub_list = [x[0:-1].split('\t')[0] for x in sub_list_raw] # remove 
    age_list = [x[0:-1].split('\t')[1] for x in sub_list_raw]
    SUB_LIST.append(sub_list);  AGE_LIST.append(age_list);
    N_sub=len(sub_list)
    print(N_sub, sub_list, age_list)
    DATA_DIR='/data/'+current_group
    TMPT_list=['desc-preproc_T1w.nii.gz', \
               'desc-brain_mask.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz', \
               'space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'];
    print('Masking ',current_group,' group starts:')
    t0=time.time()
    for i_sub in range(0, N_sub,1):
        sub_str=sub_list[i_sub]
        print('Masking ',sub_str,' ( ',i_sub , ' )...')
        res1 = dataGraber_sub(sub_str, DATA_DIR, TMPT_list)
        ind_t1 = res1[0]; ind_bm = res1[1];
        tmp_t1 = res1[2]; tmp_bm = res1[3];
        ind_masked_t1 = current_ind_OUT_DIR+sub_str+'_'+'desc-masked_T1w.nii.gz'
        tmp_masked_t1 = current_tmp_OUT_DIR+sub_str+'_'+'space-MNI152NLin2009cAsym_desc-masked_T1w.nii.gz'
        mask_ind = fsl.ApplyMask(
            in_file=ind_t1,
            out_file=ind_masked_t1,
            mask_file=ind_bm)
        mask_ind.run()
        mask_tmp = fsl.ApplyMask(
            in_file=tmp_t1,
            out_file=tmp_masked_t1,
            mask_file=tmp_bm)
        mask_tmp.run()
    print('Masking ',current_group,' finished in ',time.time()-t0, '\n')

201009-22:26:13,282 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0
40 ['sub-0002', 'sub-0004', 'sub-0005', 'sub-0006', 'sub-0008', 'sub-0009', 'sub-0012', 'sub-0014', 'sub-0015', 'sub-0021', 'sub-0022', 'sub-0023', 'sub-0024', 'sub-0025', 'sub-0028', 'sub-0030', 'sub-0031', 'sub-0034', 'sub-0035', 'sub-0037', 'sub-0038', 'sub-0040', 'sub-0047', 'sub-0051', 'sub-0052', 'sub-0068', 'sub-0075', 'sub-0076', 'sub-0094', 'sub-0096', 'sub-0098', 'sub-0109', 'sub-0111', 'sub-0118', 'sub-0125', 'sub-0129', 'sub-0132', 'sub-0136', 'sub-1000', 'sub-1020'] ['70', '76', '45', '63', '59', '57', '47', '66', '65', '53', '62', '50', '63', '44', '61', '72', '51', '70', '69', '68', '54', '63', '75', '54', '56', '66', '66', '48', '66', '73', '57', '69', '69', '54', '60', '69', '74', '62', '77', '6']
Masking  PD  group starts:
Masking  sub-0002  (  0  )...
Grabbing files for:  sub-0002
dataGraber takes:  0.016872406005859375
Masking  sub-0004  (  1  )...
Gr